In [2]:
from math import *

x, y = 3, 4
dist = sqrt(x**2 + y**2)
height = x*sin(acos(x/dist)/x)
{
    "x": x,
    "y": y,
    "dist": dist,
    "height": height,
    "contrib": height/dist
}

{'x': 3,
 'y': 4,
 'dist': 5.0,
 'height': 0.912599683022725,
 'contrib': 0.182519936604545}

In [1]:
from sympy import *
from sympy.abc import *
from sympy.parsing import parse_expr


def solve_triangle(
    # sides
    left_width=None,
    right_width=None,
    total_width=None,
    left_side=None,
    right_side=None,
    height=None,
    # angles
    top_left=None,
    top_right=None,
    top_center=None,
    bottom_left=None,
    bottom_right=None,
):
    """
    Solves the following triangle for any known values

                 /|\ top_center (top_right + top_left)
                / | /\ <--top_right
    top_left-->/\ |/   \
              /  \|      \
    left_side/    |        \
            /     |          \
           /      |            \ right_side
          /       |              \
         /        |height          \
        /         |                  \
       /          |                    \
      /           |                      \
     /\bottom_left|                       /\ <--- bottom_right
    /  \<-/       |                      /   \
    ____\_______________________________/______
    <-left_width-> <------right_width--------->
    <-----------------total_width------------->
    """
    # v for values or vars
    orig = {
        # sides
        'left_width': left_width,
        'right_width': right_width,
        'total_width': total_width,
        'left_side': left_side,
        'right_side': right_side,
        'height': height,
        # angles
        'top_left': top_left,
        'top_right': top_right,
        'top_center': top_center,
        'bottom_left': bottom_left,
        'bottom_right': bottom_right,
    }
    v = orig.copy()
    have = lambda *args: all([v.get(k) is not None for k in args])
    def set(key, value):
        if v.get(key) is None:
            v[key] = value

    # Calculate any of the obviously missing things
    if have('bottom_left',  'bottom_right'): set('top_center',    v['bottom_left'] + v['bottom_right'])
    if have('bottom_left',  'top_center'):   set('bottom_right',  v['top_center']  - v['bottom_left'])
    if have('bottom_right', 'top_center'):   set('bottom_left',   v['top_center']  - v['bottom_right'])

    if have('left_width',  'right_width'):   set('total_width',   v['left_width']   + v['right_width'])
    if have('total_width', 'left_width'):    set('right_width',   v['total_width']  - v['left_width'])
    if have('total_width', 'right_width'):   set('left_width',    v['total_width']  - v['right_width'])

    # All triangles must contain 180 degrees
    # 90 degrees
    v['square'] = pi/2
    triangle_angles = (
        ('top_center', 'bottom_right', 'bottom_left'),
        ('bottom_right', 'top_right', 'square'),
        ('bottom_left', 'top_left', 'square'),
    )
    for a, b, c in triangle_angles:
        if have(a, b): set(c, pi - v[a] - v[b])
        if have(a, c): set(b, pi - v[a] - v[c])
        if have(b, c): set(a, pi - v[b] - v[c])
    del v['square']

    # Trig stuff
    angleRelations = (
        ('bottom_right', 'height',      'right_width', 'right_side'),
        ('top_right',    'right_width', 'height',      'right_side'),
        ('bottom_left',  'height',      'left_width',  'left_side'),
        ('top_left',     'left_width',  'height',      'left_side'),
    )
    for ang, opp, adj, hyp in angleRelations:
        # Soh Cah Toa
        if have(opp, hyp): set(ang, asin(v[opp] / v[hyp]))
        if have(adj, hyp): set(ang, acos(v[adj] / v[hyp]))
        if have(opp, adj): set(ang, atan(v[opp] / v[adj]))

        if have(ang):
            theta = v[ang]
            if have(hyp):
                set(opp, v[hyp] * sin(theta))
                set(adj, v[hyp] * cos(theta))
            if have(opp):
                set(hyp, v[opp] / sin(theta))
                # AI says this should be cos?
                set(adj, v[opp] / tan(theta))
            if have(adj):
                set(hyp, v[adj] / cos(theta))
                set(opp, v[adj] * tan(theta))

    # Pythagorean theorem
    right_triangle_lines = (
        ('right_side', 'right_width', 'height'),
        ('left_side',  'left_width',  'height'),
    )
    for a, b, c in right_triangle_lines:
        if have(a, b): set(c, sqrt(v[a]**2 + v[b]**2))
        if have(a, c): set(b, sqrt(v[a]**2 + v[c]**2))
        if have(b, c): set(a, sqrt(v[b]**2 + v[c]**2))



    # Law of cosines
    if have('total_width', 'left_side', 'right_side'):
        a = v['left_side']
        b = v['right_side']
        c = v['total_width']
        set('top_left', acos((b**2 + c**2 - a**2) / (2 * b * c)))
        set('top_right', acos((a**2 + c**2 - b**2) / (2 * a * c)))
        set('top_center', acos((a**2 + b**2 - c**2) / (2 * a * b)))



    # If we have the width and height, we can find the angles
    # if have(left_width, right_width, height):
    #     # AI generated this, it may be suspect
    #     calc('top_left', asin(parse(height) / parse(left_width)))
    #     calc('top_right', asin(parse(height) / parse(right_width)))
    #     calc('bottom_left', pi - parse(top_left) - parse(top_right))
    #     calc('bottom_right', pi - parse(top_left) - parse(top_right))

    # If we have all 3 sides, we can find the angles
    # if have(left_side, right_side, total_width):
        # AI generated this, it may be suspect
        # have('left_width',   v['left_side']  / v['total_width'])
        # have('right_width',  v['right_side'] / v['total_width'])
        # have('height',       sqrt(v['left_width']**2 + v['right_width']**2))
        # have('top_left',     asin(v['left_width'] / v['height']))
        # have('top_right',    asin(v['right_width'] / v['height']))
        # have('bottom_left',  pi - v['top_left'] - v['top_right'])
        # have('bottom_right', pi - v['top_left'] - v['top_right'])

    #* If we have top_center and a line adjacent to it, we can make an imaginary triangle on the outside,
    #   and then solve for that triangle's opposite (height) and adjecent (base)
    # imaginaryOutsideTriangles = (
    #     (right_side, right_width),
    #     (_left_side, left_width)
    # )
    # if have(top_center):
    #     for hyp, base in imaginaryOutsideTriangles:
    #         if have(hyp):
    #             theta = rad(180) - top_center
    #             # print('theta:', theta)
    #             # print('hyp:', hyp)
    #             # print('cos(theta):', cos(theta))
    #             # print('sin(theta):', sin(theta))
    #             calc(base, hyp * cos(theta))
    #             calc(height, hyp * sin(theta))

    # Keep solving based on what we just did until either can't solve any more or we have all the values
    if orig != v and not all(orig.values()):
        return solve_triangle(**v)

    return v

import math
var('opp, adj, hyp, x, y, dist')
# solve_triangle(left_side='y', right_side='x', total_width='dist', top_center=pi/2)
solve_triangle(left_side=y, right_side=x, total_width=dist)

# solve_triangle(right_width=adj, height=opp)
# solve_triangle(right_width=5.395, height=3.077)

<>:21: SyntaxWarning: invalid escape sequence '\ '
<>:21: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipykernel_1201023/2108531126.py:21: SyntaxWarning: invalid escape sequence '\ '
  """


{'left_width': y*cos(pi/2 - acos((dist**2 + x**2 - y**2)/(2*dist*x))),
 'right_width': x*cos(pi/2 - acos((dist**2 - x**2 + y**2)/(2*dist*y))),
 'total_width': dist,
 'left_side': y,
 'right_side': x,
 'height': x*sin(pi/2 - acos((dist**2 - x**2 + y**2)/(2*dist*y))),
 'top_left': acos((dist**2 + x**2 - y**2)/(2*dist*x)),
 'top_right': acos((dist**2 - x**2 + y**2)/(2*dist*y)),
 'top_center': acos((-dist**2 + x**2 + y**2)/(2*x*y)),
 'bottom_left': pi/2 - acos((dist**2 + x**2 - y**2)/(2*dist*x)),
 'bottom_right': pi/2 - acos((dist**2 - x**2 + y**2)/(2*dist*y))}

In [ ]:
# Tests

# Confirmed correct triangle to 3 decimal places
tri1 = {
    'top_center': 90.002,
    'bottom_left': 29.7,
    'bottom_right': 60.299,
    'left_side': 5.395,
    'right_side': 3.077,
    'total_width': 6.211,
    'height': 2.673,
    'left_width': 4.686,
    'right_width': 1.525,
    'top_left': abs(math.pi/2 - 29.7),
    'top_right': abs(math.pi/2 - 60.299),
}

def about(a, b):
    for k in a:
        if not abs(a[k] - b[k]) < 0.001:
            return False
    return True

assert about(tri1, solve_triangle())


TypeError: unsupported operand type(s) for -: 'float' and 'NoneType'

In [ ]:
# Bad ai generated tests
def test_solve_triangle():
    # Test Case 1: Right triangle (3-4-5)
    tri1 = {
        'x': 3,
        'y': 4,
        'dist': 5.0,
        'height': 0.912599683022725,
        'contrib': 0.182519936604545
    }
    result1 = solve_triangle(x=3, y=4)
    assert about(tri1, result1), "3-4-5 right triangle test failed"

    # Test Case 2: Equilateral triangle
    result2 = solve_triangle(a=60, b=60, c=60, side_ab=1)
    assert about(1, result2['side_bc']), "Equilateral triangle side test failed"
    assert about(1, result2['side_ac']), "Equilateral triangle side test failed"

    # Test Case 3: Isosceles triangle
    result3 = solve_triangle(side_ab=5, side_ac=5, angle_b=60)
    assert about(5, result3['side_bc']), "Isosceles triangle test failed"
    assert about(60, result3['angle_c']), "Isosceles triangle angle test failed"

    # Test Case 4: Scalene triangle with angles
    result4 = solve_triangle(angle_a=30, angle_b=60, side_ab=2)
    assert about(90, result4['angle_c']), "Scalene triangle angle test failed"
    assert about(1, result4['side_bc']), "30-60-90 triangle side test failed"

    # Test Case 5: SSS (Side-Side-Side) triangle
    result5 = solve_triangle(side_ab=7, side_bc=8, side_ac=9)
    assert about(73.4, result5['angle_a'], tolerance=0.1), "SSS triangle angle A test failed"
    assert about(48.2, result5['angle_b'], tolerance=0.1), "SSS triangle angle B test failed"

    # Test Case 6: SAS (Side-Angle-Side) triangle
    result6 = solve_triangle(side_ab=5, angle_c=60, side_bc=8)
    assert about(7, result6['side_ac'], tolerance=0.1), "SAS triangle side test failed"

    # Test Case 7: ASA (Angle-Side-Angle) triangle
    result7 = solve_triangle(angle_a=45, side_ab=10, angle_b=60)
    assert about(75, result7['angle_c']), "ASA triangle angle test failed"
    assert about(8.97, result7['side_bc'], tolerance=0.1), "ASA triangle side test failed"

    # Test Case 8: AAS (Angle-Angle-Side) triangle
    result8 = solve_triangle(angle_a=30, angle_b=60, side_bc=10)
    assert about(20, result8['side_ab'], tolerance=0.1), "AAS triangle side test failed"

    # Test Case 9: Right triangle with hypotenuse and one side
    result9 = solve_triangle(side_ab=3, side_bc=5, angle_a=90)
    assert about(4, result9['side_ac'], tolerance=0.1), "Right triangle with hypotenuse test failed"

    # Test Case 10: Triangle with area calculation
    result10 = solve_triangle(side_ab=5, side_ac=6, angle_a=30)
    area = 0.5 * 5 * 6 * sin(radians(30))
    assert about(area, result10['area']), "Triangle area calculation test failed"

    print("All tests passed!")

# Helper function for floating point comparison
def about(expected, actual, tolerance=1e-6):
    if isinstance(expected, dict) and isinstance(actual, dict):
        if set(expected.keys()) != set(actual.keys()):
            print(f"Key mismatch: expected {expected.keys()}, got {actual.keys()}")
            return False
        return all(about(expected[k], actual[k], tolerance) for k in expected)
    elif isinstance(expected, (int, float)) and isinstance(actual, (int, float)):
        return abs(expected - actual) < tolerance
    else:
        return expected == actual

test_solve_triangle()

TypeError: solve_triangle() got an unexpected keyword argument 'x'